In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
#define image size and batch size
IMG_SIZE=224
BATCH_SIZE=32

In [ ]:
#Define data generators for train, validation and test sets
train_datagen= ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)
train_generator= train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain_Tumor_Detection/Train',
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'binary',
    subset='training'
)
val_generator= train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain_Tumor_Detection/Train',
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'binary',
    subset='validation'
)
test_datagen= ImageDataGenerator(
    rescale=1./255
)

test_generator= train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain_Tumor_Detection/Test',
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'binary'
)

Found 2408 images belonging to 2 classes.
Found 602 images belonging to 2 classes.
Found 60 images belonging to 1 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#define the model
model=keras.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(1,activation='sigmoid')
])

In [ ]:
#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#Fitting the model
history=model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
76/76 [==============================] - 654s 9s/step - loss: 0.5219 - accuracy: 0.7762 - val_loss: 0.3230 - val_accuracy: 0.8721
Epoch 2/5
76/76 [==============================] - 278s 4s/step - loss: 0.2363 - accuracy: 0.9124 - val_loss: 0.1390 - val_accuracy: 0.9518
Epoch 3/5
76/76 [==============================] - 284s 4s/step - loss: 0.1381 - accuracy: 0.9522 - val_loss: 0.1188 - val_accuracy: 0.9668
Epoch 4/5
76/76 [==============================] - 299s 4s/step - loss: 0.0667 - accuracy: 0.9801 - val_loss: 0.0318 - val_accuracy: 0.9884
Epoch 5/5
76/76 [==============================] - 324s 4s/step - loss: 0.0268 - accuracy: 0.9921 - val_loss: 0.0312 - val_accuracy: 0.9917


In [ ]:
#saving the model
model.save("Model.h5","label.txt")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

#Load the saved model
model =load_model('/content/Model.h5')

#Load and preprocess the test image
test_image_path='/content/drive/MyDrive/Brain_Tumor_Detection/Test/pred/pred27.jpg'
img=image.load_img(test_image_path, target_size=(224,224))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array,axis=0)#add batch dimension
img_array/=255. #Normalize the pixel values

#make predictions
prediction=model.predict(img_array)

#print the predictions
if prediction < 0.5:
  print("Prediction: No Tumor (Probability:",prediction[0][0],")")
else:
  print("Prediction: Tumor present (Probability:",prediction[0][0],")")

1/1 [==============================] - 0s 112ms/step
Prediction: Tumor present (Probability: 0.99999917 )
